In [1]:
# Import des librairies
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler, LabelEncoder, OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import matplotlib.pyplot as plt
import seaborn as sns

# Chargement du fichier CSV
df = pd.read_csv("merged_data.csv")

# Afficher les premières lignes pour vérifier
df.head()


,_id_x,genre,age,nationalite,niveauEtudes,sessionId,__v_x,_id_y,textId,associationType,personAType,valueOneA,valueTwoA,forceA,personBType,valueOneB,valueTwoB,forceB,createdAt,__v_y
0,67e05b30dacaf24880d98fe8,homme,23,HRV,bac+4,420008aa-5136-48fb-9a6f-4347673fce87,0,67e05bb9dacaf24880d98feb,67a243ea6d8e29a001947946,risk-reward,Robot,8,5,8,Personne âgée,2,5,3,2025-03-23T19:06:33.515Z,0
1,67e05b30dacaf24880d98fe8,homme,23,HRV,bac+4,420008aa-5136-48fb-9a6f-4347673fce87,0,67e05c02dacaf24880d98fed,67a243ea6d8e29a001947945,risk-reward,Femme petite taille,4,7,5,Robot,6,3,8,2025-03-23T19:07:46.725Z,0
2,67e05b30dacaf24880d98fe8,homme,23,HRV,bac+4,420008aa-5136-48fb-9a6f-4347673fce87,0,67e05c25dacaf24880d98fef,67a51efa18a0dfe14aac016b,risk-effort,Enfant,5,3,3,Femme petite taille,5,7,5,2025-03-23T19:08:21.185Z,0
3,67e05b30dacaf24880d98fe8,homme,23,HRV,bac+4,420008aa-5136-48fb-9a6f-4347673fce87,0,67e05c33dacaf24880d98ff1,67a243ea6d8e29a001947947,effort-reward,Enfant,3,3,3,Femme petite taille,7,7,5,2025-03-23T19:08:35.846Z,0
4,67e05b30dacaf24880d98fe8,homme,23,HRV,bac+4,420008aa-5136-48fb-9a6f-4347673fce87,0,67e05c4bdacaf24880d98ff3,67a243ea6d8e29a001947949,effort-reward,Femme grande taille,5,5,7,Homme petite taille,5,5,6,2025-03-23T19:08:59.087Z,0


In [15]:
# Création des colonnes cibles initialisées à NaN
df["effort"] = None
df["risque"] = None
df["reward"] = None

def remplir_valeurs(row):
    # Initialiser les cibles
    row["effort"] = None
    row["risque"] = None
    row["reward"] = None

    type_assoc = row["associationType"]
    
    # Récupération des valeurs
    v1a = row["valueOneA"]
    v2a = row["valueTwoA"]
    v1b = row["valueOneB"]
    v2b = row["valueTwoB"]
    
    # Cas effort
    if "effort" in type_assoc:
        row["effort"] = (v1a + v1b) / 2

    # Cas risque (risk)
    if "risk" in type_assoc:
        row["risque"] = (v1a + v1b) / 2

    # Cas reward
    if "reward" in type_assoc:
        row["reward"] = (v2a + v2b) / 2

    return row


# Appliquer à chaque ligne
df = df.apply(remplir_valeurs, axis=1)

# Vérif rapide
df[["associationType", "effort", "risque", "reward"]].head()

,associationType,effort,risque,reward
0,risk-reward,NaN,5.0,5.0
1,risk-reward,NaN,5.0,5.0
2,risk-effort,5.0,5.0,NaN
3,effort-reward,5.0,NaN,5.0
4,effort-reward,5.0,NaN,5.0


In [6]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestRegressor
from sklearn.multioutput import MultiOutputRegressor

# Choix des features
X = df[["genre", "age", "nationalite", "niveauEtudes", "personAType", "personBType"]]
y = df[["effort", "risque", "reward"]].astype(float)  # Cibles numériques

# Encodage + normalisation
preprocessor = ColumnTransformer(transformers=[
    ("num", StandardScaler(), ["age"]),
    ("cat", OneHotEncoder(handle_unknown="ignore"), ["genre", "nationalite", "niveauEtudes", "personAType", "personBType"])
])

print(X)


      genre  age nationalite niveauEtudes          personAType  \
0     homme   23         HRV        bac+4                Robot   
1     homme   23         HRV        bac+4  Femme petite taille   
2     homme   23         HRV        bac+4               Enfant   
3     homme   23         HRV        bac+4               Enfant   
4     homme   23         HRV        bac+4  Femme grande taille   
...     ...  ...         ...          ...                  ...   
1255  homme   22         GBR        bac+2               Enfant   
1256  homme   22         GBR        bac+2               Enfant   
1257  homme   22         GBR        bac+2  Femme petite taille   
1258  homme   22         GBR        bac+2                Robot   
1259  homme   21         JPN        bac+1                Robot   

              personBType  
0           Personne âgée  
1                   Robot  
2     Femme petite taille  
3     Femme petite taille  
4     Homme petite taille  
...                   ...  
1255  Homme

In [8]:
# Vérifie combien de lignes contiennent au moins un NaN dans les cibles
print(y.isnull().sum())
print("Nombre total de lignes avec au moins une cible manquante :", y.isnull().any(axis=1).sum())


effort    416
risque    434
reward    410
dtype: int64
Nombre total de lignes avec au moins une cible manquante : 1260


In [7]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.multioutput import MultiOutputRegressor

# Split des données
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Pipeline complet
pipeline = Pipeline(steps=[
    ("preprocessing", preprocessor),
    ("model", MultiOutputRegressor(RandomForestRegressor(n_estimators=100, random_state=42)))
])

# Entraîner le modèle
pipeline.fit(X_train, y_train)


ValueError: Input y contains NaN.

In [11]:
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import numpy as np

def entrainer_modele_pour_cible(df, cible):
    print(f"\n Modèle pour : {cible.upper()}")
    
    # Garder uniquement les lignes où la cible n’est pas NaN
    data = df[df[cible].notna()]
    
    # Features (mêmes que toi)
    X = data[["genre", "age", "nationalite", "niveauEtudes", "personAType", "personBType"]]
    y = data[cible].astype(float)
    
    # Pipeline de prétraitement
    preprocessor = ColumnTransformer(transformers=[
        ("num", StandardScaler(), ["age"]),
        ("cat", OneHotEncoder(handle_unknown="ignore"), ["genre", "nationalite", "niveauEtudes", "personAType", "personBType"])
    ])
    
    pipeline = Pipeline(steps=[
        ("preprocessing", preprocessor),
        ("model", RandomForestRegressor(n_estimators=100, random_state=42))
    ])
    
    # Split train/test
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    
    # Entraînement
    pipeline.fit(X_train, y_train)
    
    # Prédictions
    y_pred = pipeline.predict(X_test)
    
    # Évaluation
    print("MAE :", mean_absolute_error(y_test, y_pred))
    mse = mean_squared_error(y_test, y_pred)
    rmse = np.sqrt(mse)
    print("RMSE :", rmse)
    print("R² :", r2_score(y_test, y_pred))
    
    return pipeline


In [12]:
modele_effort = entrainer_modele_pour_cible(df, "effort")
modele_risque = entrainer_modele_pour_cible(df, "risque")
modele_reward = entrainer_modele_pour_cible(df, "reward")



🎯 Modèle pour : EFFORT
MAE : 0.0
RMSE : 0.0
R² : 1.0

🎯 Modèle pour : RISQUE
MAE : 0.0
RMSE : 0.0
R² : 1.0

🎯 Modèle pour : REWARD
MAE : 0.0
RMSE : 0.0
R² : 1.0


In [13]:
print("Nombre de lignes avec effort :", df["effort"].notna().sum())
print("Nombre de lignes avec risque :", df["risque"].notna().sum())
print("Nombre de lignes avec reward :", df["reward"].notna().sum())


Nombre de lignes avec effort : 844
Nombre de lignes avec risque : 826
Nombre de lignes avec reward : 850


In [14]:
nouveau_participant = pd.DataFrame([{
    "genre": "femme",
    "age": 22,
    "nationalite": "FRA",
    "niveauEtudes": "bac+3",
    "personAType": "Robot",
    "personBType": "Personne âgée"
}])

# Prédire effort
pred_effort = modele_effort.predict(nouveau_participant)
print("Effort prédit :", pred_effort[0])


Effort prédit : 5.0
